<a href="https://colab.research.google.com/github/aBaldoqui/ImageSearch/blob/main/buscaporimagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import models, transforms
from PIL import Image

# Carregar o modelo pré-treinado
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remover a última camada de classificação
model.eval()

# Transformação da imagem
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_embedding(image_path):
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)  # Adiciona batch dimension
    with torch.no_grad():
        embedding = model(image).squeeze()  # Remove batch dimension
    return embedding.numpy()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 33.5MB/s]


In [3]:
import numpy as np

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    return dot_product / (norm1 * norm2)


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 62.0 MB/s eta 0:00:00


In [4]:
import os
# Definir o caminho da pasta 'roupas'
image_dir = '/content/roupas'

# Listar todos os arquivos .jpg na pasta 'roupas'
image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith('.jpg')]

# Extrair os embeddings e armazená-los na lista 'embeddings'
embeddings = []
for image_path in image_paths:
    emb = extract_embedding(image_path)
    embeddings.append(emb)

# Converter a lista de embeddings para um array NumPy
embeddings = np.array(embeddings)

# Mostrar a forma do array de embeddings (ex: número de imagens, dimensão do embedding)
print(embeddings.shape)

(17, 2048)


In [10]:
len(embeddings)

17

In [19]:
import faiss
import numpy as np

# Supondo que temos embeddings de todas as imagens em 'embeddings'
d = embeddings.shape[1]  # Dimensão do embedding
index = faiss.IndexFlatL2(d)  # Indexador de similaridade com L2 (euclidiana)
index.add(embeddings)  # Adicionar os embeddings ao indexador

# Buscar imagem mais similar
D, I = index.search(np.array([embeddings[11]]), k=5)  # Retorna os 5 mais similares

print(I)

[[11 10  3  4 16]]


In [21]:
image_paths[3]

'/content/roupas/3094ba16a14e554292846f4551169788.jpg'